# Busca informações da ocorrência policiais dos municípios do Estado de São Paulo

## Programa Mentoria DSA

* Autor: Rogelio Gonçaves Neto
* Apoio: Luciana Silva Albuquerque de Melo e Leticia Fiorina Echeverria Grobério
* Projeto: Análise da criminalidade do estado de São Paulo relacionada a Educação e Renda
* Objetivo: Criar um dataset com os dados das ocorrências policiais para realização do projeto de mentoria da DSA.
* Necessidades: chromedrive.exe conforme versão do navegador instalado
* Bibliotecas: time, selenium, shutil, pandas

## Observações

* Todos os Dataset de apoio gerados ficam armazenado em **C:\Mentoria\Dataset\Temporario**
* Os diretorios para download devem ser ajustados: **C://Users//DELL//Downloads//**
* Este dataset publico contem o ano de 2020

## Resultado Final
* Será gerado no diretorio **C:\Mentoria\Dataset\Temporario** os arquivos:  
    **Geracao.xlsx**, Este arquivo controla o id dos municípios caso o programa abort é so reiniciar a **Etapa 01**  
    **ProdutividadePolicial- + NOME_DO_MUNICIPIO.csv**, Será gerando um arquivo para cada município do Estado de São Paulo  
    **PRODUTIVIDADE_POLICIAL.xlsx**, Arquivo com os dados em linha por tipo de ocorrencia  
    **Observo que:** Estes arquivos contem dados que podem ser utilizados para maiores detalhes para exploração, porém devido ao tempo foi realizado a geração de um arquivo com **resumo** dos dados para finalização do trabalho proposto.  
    
* Foi gerado **um** arquivo no diretorio **C:\Mentoria\Dataset**
    Arquivo final para ser utilizado no trabalho final para as entregas  
    OCOR_MUNICIPIO.xlsx

In [ ]:
import time as t
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import shutil
import pandas as pd
from pathlib import Path

In [ ]:
df_munic = pd.read_excel('C:\Mentoria\Dataset\PIB_MUNICIPIO.xlsx', usecols = "C")
df_munic = df_munic.drop_duplicates()
df_munic

In [ ]:
df_geracao = pd.DataFrame(columns=['Ano','Sequencia'])
df_geracao.loc[0] = [int(2020),int(1)]

In [ ]:
df_geracao.to_excel(r'C://Mentoria//Dataset//Temporario//Geracao.xlsx')

In [ ]:
teste = 'SIM'

In [ ]:
# Etapa 01 - Baixar arquivos do site da policia
# =============================================
ult_seq = int(df_geracao.Sequencia)
ano     = int(df_geracao.Ano)
if teste == 'SIM':
    tot_munic = 3
else:
    tot_munic = len(df_munic.Municípios)
    
for m in range(0, tot_munic, 1):
    print('Sequencia: ' + str(m+int(ult_seq)))
    if m+ult_seq >= tot_munic:
        exit

    df_geracao.loc[0] = [ano, m+ult_seq]
    df_geracao.loc[0] = [ano, m+ult_seq]

    t.sleep(2)
    navegador = webdriver.Chrome(executable_path=r'C:/Mentoria/Dataset/chromedriver.exe')
    t.sleep(2)
    navegador.get("https://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx")
    t.sleep(2)
    navegador.find_element_by_xpath('//*[@id="conteudo_ddlAnos"]').send_keys(int(ano)) #df_geracao.Ano))  #ano)
    navegador.find_element_by_xpath('//*[@id="conteudo_ddlRegioes"]').send_keys('Todos')
    municipio = '//*[@id="conteudo_ddlMunicipios"]/option[' + str(m+ult_seq) + ']'
    t.sleep(2)
    navegador.find_element_by_xpath(municipio).click()
    navegador.find_element_by_xpath('//*[@id="conteudo_ddlDelegacias"]').send_keys('Todos')
    t.sleep(5)    
    navegador.find_element_by_xpath('//*[@id="conteudo_btnExcel"]').click()  
    t.sleep(3)    
    navegador.close()    

In [ ]:
for filename in Path(r'C://Users//DELL//Downloads//').rglob("ProdutividadePolicial-*.csv"):
    print(filename.name)
    origem = r'C://Users//DELL//Downloads//' + filename.name
    destino = r'C://Mentoria//Dataset//Temporario//' + filename.name 
    shutil.move(origem,destino)    

In [ ]:
## Ok
df_geral = pd.DataFrame()
ano = int(df_geracao.Ano)
for filename in Path(r'C://Mentoria//Dataset//Temporario//').rglob("ProdutividadePolicial-*.csv"):    
    df_lido = pd.read_csv(r'C://Mentoria//Dataset//Temporario//'+filename.name, delimiter=";", encoding='UTF-16 LE', skiprows=1)                     

    df_lido.insert(0,"Ano",ano,True)
    municipio = filename.name
    municipio = municipio.replace("ProdutividadePolicial-Estado de ","").replace("ProdutividadePolicial-Município ","").replace(".csv","").replace("-2020","")
    df_lido.insert(0,"Municipio",municipio,True)
    
    if df_geral.empty:
        df_geral = df_lido 
    else:
        df_geral = pd.concat([df_geral, df_lido])

df_geral.Municipio = df_geral.Municipio.str.upper()
df_geral.Municipio = df_geral.Municipio.str.replace('Á','A').str.replace('Í','I').str.replace('Ó','O')
df_geral.Municipio = df_geral.Municipio.str.replace('Ã','A').str.replace('É','E').str.replace('Â','A')
df_geral.Municipio = df_geral.Municipio.str.replace('Ê','E').str.replace('Ô','O').str.replace('Ç','C')
df_geral.Municipio = df_geral.Municipio.str.replace('Ú','U').str.replace('Õ','O')

df_geral.Ocorrencia = df_geral.Ocorrencia.str.upper()
df_geral.Ocorrencia = df_geral.Ocorrencia.str.replace('Á','A').str.replace('Í','I').str.replace('Ó','O')
df_geral.Ocorrencia = df_geral.Ocorrencia.str.replace('Ã','A').str.replace('É','E').str.replace('Â','A')
df_geral.Ocorrencia = df_geral.Ocorrencia.str.replace('Ê','E').str.replace('Ô','O').str.replace('Ç','C')
df_geral.Ocorrencia = df_geral.Ocorrencia.str.replace('Ú','U').str.replace('Õ','O').str.replace('º','.')
df_geral = df_geral.drop(columns=['Unnamed: 14'])

df_geral.to_excel(r'C://Mentoria//Dataset//Temporario//PRODUTIVIDADE_POLICIAL.xlsx', encoding='UTF-8')                          

In [ ]:
OcorMunicipio = pd.read_excel(r'C:\Mentoria\Dataset\Temporario\PRODUTIVIDADE_POLICIAL.xlsx',index_col=0)
OcorMunicipio = OcorMunicipio.drop(columns=['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez'])

OcorMunicipio1 = OcorMunicipio.query('Ocorrencia=="OCORRENCIAS DE PORTE DE ENTORPECENTES"')
OcorMunicipio1 = OcorMunicipio1.drop(columns=['Ocorrencia'])
OcorMunicipio1 = OcorMunicipio1.rename(columns={'Total': 'OCORRENCIAS DE PORTE DE ENTORPECENTES'})
                                                
OcorMunicipio2 = OcorMunicipio.query('Ocorrencia=="OCORRENCIAS DE TRAFICO DE ENTORPECENTES"')
OcorMunicipio2 = OcorMunicipio2.drop(columns=['Ocorrencia'])
OcorMunicipio2 = OcorMunicipio2.rename(columns={'Total': 'OCORRENCIAS DE TRAFICO DE ENTORPECENTES'})
dataRes = pd.merge(OcorMunicipio1, OcorMunicipio2, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio3 = OcorMunicipio.query('Ocorrencia=="OCORRENCIAS DE APREENSAO DE ENTORPECENTES(1)"')
OcorMunicipio3 = OcorMunicipio3.drop(columns=['Ocorrencia'])
OcorMunicipio3 = OcorMunicipio3.rename(columns={'Total': 'OCORRENCIAS DE APREENSAO DE ENTORPECENTES(1)'})
dataRes = pd.merge(dataRes, OcorMunicipio3, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio4 = OcorMunicipio.query('Ocorrencia=="OCORRENCIAS DE PORTE ILEGAL DE ARMA"')
OcorMunicipio4 = OcorMunicipio4.drop(columns=['Ocorrencia'])
OcorMunicipio4 = OcorMunicipio4.rename(columns={'Total': 'OCORRENCIAS DE PORTE ILEGAL DE ARMA'})
dataRes = pd.merge(dataRes, OcorMunicipio4, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio5 = OcorMunicipio.query('Ocorrencia=="N. DE ARMAS DE FOGO APREENDIDAS"')
OcorMunicipio5 = OcorMunicipio5.drop(columns=['Ocorrencia'])
OcorMunicipio5 = OcorMunicipio5.rename(columns={'Total': 'N. DE ARMAS DE FOGO APREENDIDAS'})
dataRes = pd.merge(dataRes, OcorMunicipio5, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio6 = OcorMunicipio.query('Ocorrencia=="N. DE FLAGRANTES LAVRADOS"')
OcorMunicipio6 = OcorMunicipio6.drop(columns=['Ocorrencia'])
OcorMunicipio6 = OcorMunicipio6.rename(columns={'Total': 'N. DE FLAGRANTES LAVRADOS'})
dataRes = pd.merge(dataRes, OcorMunicipio6, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio7 = OcorMunicipio.query('Ocorrencia=="N. DE INFRATORES APREENDIDOS EM FLAGRANTE"')
OcorMunicipio7 = OcorMunicipio7.drop(columns=['Ocorrencia'])
OcorMunicipio7 = OcorMunicipio7.rename(columns={'Total': 'N. DE INFRATORES APREENDIDOS EM FLAGRANTE'})
dataRes = pd.merge(dataRes, OcorMunicipio7, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio8 = OcorMunicipio.query('Ocorrencia=="N. DE INFRATORES APREENDIDOS POR MANDADO"')
OcorMunicipio8 = OcorMunicipio8.drop(columns=['Ocorrencia'])
OcorMunicipio8 = OcorMunicipio8.rename(columns={'Total': 'N. DE INFRATORES APREENDIDOS POR MANDADO'})
dataRes = pd.merge(dataRes, OcorMunicipio8, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio9 = OcorMunicipio.query('Ocorrencia=="N. DE PESSOAS PRESAS EM FLAGRANTE"')
OcorMunicipio9 = OcorMunicipio9.drop(columns=['Ocorrencia'])
OcorMunicipio9 = OcorMunicipio9.rename(columns={'Total': 'N. DE PESSOAS PRESAS EM FLAGRANTE'})
dataRes = pd.merge(dataRes, OcorMunicipio9, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio10 = OcorMunicipio.query('Ocorrencia=="N. DE PESSOAS PRESAS POR MANDADO"')
OcorMunicipio10 = OcorMunicipio10.drop(columns=['Ocorrencia'])
OcorMunicipio10 = OcorMunicipio10.rename(columns={'Total': 'N. DE PESSOAS PRESAS POR MANDADO'})
dataRes = pd.merge(dataRes, OcorMunicipio10, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio11 = OcorMunicipio.query('Ocorrencia=="N. DE PRISOES EFETUADAS"')
OcorMunicipio11 = OcorMunicipio11.drop(columns=['Ocorrencia'])
OcorMunicipio11 = OcorMunicipio11.rename(columns={'Total': 'N. DE PRISOES EFETUADAS'})
dataRes = pd.merge(dataRes, OcorMunicipio11, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio12 = OcorMunicipio.query('Ocorrencia=="N. DE VEICULOS RECUPERADOS"')
OcorMunicipio12 = OcorMunicipio12.drop(columns=['Ocorrencia'])
OcorMunicipio12 = OcorMunicipio12.rename(columns={'Total': 'N. DE VEICULOS RECUPERADOS'})
dataRes = pd.merge(dataRes, OcorMunicipio12, how = 'left', on = ['Municipio','Ano'])

OcorMunicipio13 = OcorMunicipio.query('Ocorrencia=="TOT. DE INQUERITOS POLICIAIS INSTAURADOS"')
OcorMunicipio13 = OcorMunicipio13.drop(columns=['Ocorrencia'])
OcorMunicipio13 = OcorMunicipio13.rename(columns={'Total': 'TOT. DE INQUERITOS POLICIAIS INSTAURADOS'})
dataRes = pd.merge(dataRes, OcorMunicipio13, how = 'left', on = ['Municipio','Ano'])

dataRes['TOTAL'] = dataRes.iloc[:,2:15].sum(axis=1)
dataRes.to_excel(r"C:\Mentoria\Dataset\OCOR_MUNICIPIO.xlsx")
dataRes